# Reference Path Processing

Curvilinear coordinate frames are heavily influenced by the quality of the reference path (in our case a 2d polyline). Potential issues of the reference path are for example discontinuities, insufficient smoothness or noisy profiles of the curvature (rate), too high curvature values, oversampling etc... This can lead to a variety of problems when using the curvilinear representation for downstream tasks (e.g., motion planning), such as discontinuous trajectories, infeasibilities, arbitrary projections etc...

Therefore, we provide different methods which can be used to pre-process the reference path w.r.t. the aforementioned issues. Here, we show some examples.